- 1st grownet
https://www.kaggle.com/tmhrkt/grownet-gradient-boosting-neural-networks/data

In [1]:
import os
import sys
import random
import warnings
import numpy as np
import pandas as pd 
from sklearn import preprocessing
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.decomposition import PCA
from tqdm import tqdm_notebook as tqdm
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_selection import VarianceThreshold

sys.path.append('../input/multilabelstraifier/')
from ml_stratifiers import MultilabelStratifiedKFold
warnings.filterwarnings('ignore')

import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import tensorflow as tf
from torch.nn.modules.loss import _WeightedLoss

# install

In [2]:
DATA_DIR = '/kaggle/input/lish-moa/'
train = pd.read_csv(DATA_DIR + 'train_features.csv')
targets = pd.read_csv(DATA_DIR + 'train_targets_scored.csv')
non_targets = pd.read_csv(DATA_DIR + 'train_targets_nonscored.csv')
test = pd.read_csv(DATA_DIR + 'test_features.csv')
sub = pd.read_csv(DATA_DIR + 'sample_submission.csv')
drug = pd.read_csv(DATA_DIR + 'train_drug.csv')

target_feats = [ i for i in targets.columns if i != "sig_id"]
g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

noncons_train_index = train[train.cp_type=="ctl_vehicle"].index
cons_train_index = train[train.cp_type!="ctl_vehicle"].index
noncons_test_index = test[test.cp_type=="ctl_vehicle"].index
cons_test_index = test[test.cp_type!="ctl_vehicle"].index

# preprocess

In [3]:
train = train[train.index.isin(cons_train_index)].copy().reset_index(drop=True)
test = test[test.index.isin(cons_test_index)].copy().reset_index(drop=True)
targets = targets[targets.index.isin(cons_train_index)].copy().reset_index(drop=True)

In [4]:
NB_SPLITS = 7
seed = 34

folds = []
    
# LOAD FILES
train_score = targets.merge(drug, on='sig_id', how='left') 

# LOCATE DRUGS
vc = train_score.drug_id.value_counts()
vc1 = vc.loc[vc <= 18].index.sort_values()
vc2 = vc.loc[vc > 18].index.sort_values()
    
# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, shuffle = True, random_state = seed)
tmp = train_score.groupby('drug_id')[target_feats].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_feats])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)
    
# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits = NB_SPLITS, shuffle = True, random_state = seed)
tmp = train_score.loc[train_score.drug_id.isin(vc2)].reset_index(drop = True)
for fold,(idxT,idxV) in enumerate(skf.split(tmp,tmp[target_feats])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
train_score['fold'] = train_score.drug_id.map(dct1)
train_score.loc[train_score.fold.isna(),'fold'] = train_score.loc[train_score.fold.isna(),'sig_id'].map(dct2)
train_score.fold = train_score.fold.astype('int8')
folds.append(train_score.fold.values)
    
np.array(folds)

array([[1, 2, 6, ..., 3, 4, 0]], dtype=int8)

# feature enginnering

In [5]:
X = train.iloc[:,4:].copy().values
select = VarianceThreshold(threshold=0.7)
X_new = select.fit_transform(X)
drop_feats = list(np.array(train.iloc[:,4:].columns)[select.get_support()==False])
print(len(drop_feats))

33


In [6]:
train.drop(drop_feats, axis=1, inplace=True)
test.drop(drop_feats, axis=1, inplace=True)

g_feats = [i for i in train.columns if "g-" in i]
c_feats = [i for i in train.columns if "c-" in i]

In [7]:
# rank gauss
for i in c_feats + g_feats:
    ss = preprocessing.QuantileTransformer(n_quantiles=1000, random_state=0, output_distribution="normal")
    #    ss.fit(pd.concat([train[i], test[i]]).values.reshape(-1,1))
    ss.fit(train[i].values.reshape(-1,1))
    train[i] = ss.transform(train[i].values.reshape(-1,1))
    test[i] = ss.transform(test[i].values.reshape(-1,1))

In [8]:
c_num = 10
pca_c_cols = ["pca-c"+str(i+1) for i in range(c_num)]
pca = PCA(n_components=c_num,random_state=42)
c_train = pca.fit_transform(train[c_feats])
c_test = pca.transform(test[c_feats])
c_train = pd.DataFrame(c_train, columns=pca_c_cols)
c_test = pd.DataFrame(c_test, columns=pca_c_cols)

g_num = 60
pca_g_cols = ["pca-g"+str(i+1) for i in range(g_num)]
pca = PCA(n_components=g_num, random_state=42)
g_train = pca.fit_transform(train[g_feats])
g_test = pca.transform(test[g_feats])
g_train = pd.DataFrame(g_train, columns=pca_g_cols)
g_test = pd.DataFrame(g_test, columns=pca_g_cols)

train = pd.concat([train, c_train],axis=1)
test = pd.concat([test, c_test],axis=1)
train = pd.concat([train, g_train],axis=1)
test = pd.concat([test, g_test],axis=1)

In [9]:
def fe(df):
    tmp = df.copy()
    tmp['g_kurt'] = tmp[g_feats].kurtosis(axis = 1)
    tmp['g_skew'] = tmp[g_feats].skew(axis = 1)
    tmp['c_kurt'] = tmp[c_feats].kurtosis(axis = 1)
    tmp['c_skew'] = tmp[c_feats].skew(axis = 1)
    tmp = pd.get_dummies(tmp, columns=['cp_time','cp_dose'])
    tmp.drop(["cp_type"], axis=1, inplace=True)
    return tmp

train = fe(train)
test = fe(test)

print(train.shape, test.shape)

(21948, 919) (3624, 919)


In [10]:
feature_cols = train.columns[1:].tolist()

In [11]:
train = train.merge(targets, on ="sig_id", how="left")

In [12]:
train["fold"] = np.array(folds).reshape(-1,1)

# class

In [13]:
class MoADataset:
    def __init__(self, features, targets):
        self.features = features
        self.targets = targets
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :] , dtype=torch.float),
            'y' : torch.tensor(self.targets[idx, :], dtype=torch.float)            
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [14]:
from enum import Enum
class ForwardType(Enum):
    SIMPLE = 0
    STACKED = 1
    CASCADE = 2
    GRADIENT = 3

class DynamicNet(object):
    def __init__(self, c0, lr):
        self.models = []
        self.c0 = c0
        self.lr = lr
        self.boost_rate  = nn.Parameter(torch.tensor(lr, requires_grad=True, device="cuda"))

    def add(self, model):
        self.models.append(model)

    def parameters(self):
        params = []
        for m in self.models:
            params.extend(m.parameters())

        params.append(self.boost_rate)
        return params
    
    def zero_grad(self):
        for m in self.models:
            m.zero_grad()

    def to_cuda(self):
        for m in self.models:
            m.cuda()

    def to_eval(self):
        for m in self.models:
            m.eval()

    def to_train(self):
        for m in self.models:
            m.train(True)
            
    def forward(self, x):
        if len(self.models) == 0:
            batch = x.shape[0]
            c0 = np.repeat(self.c0.detach().cpu().numpy().reshape(1,-1), batch, axis=0)
            return None, torch.Tensor(c0).cuda()
        middle_feat_cum = None
        prediction = None
        with torch.no_grad():
            for m in self.models:
                if middle_feat_cum is None:
                    middle_feat_cum, prediction = m(x, middle_feat_cum)
                else:
                    middle_feat_cum, pred = m(x, middle_feat_cum)
                    prediction += pred
        return middle_feat_cum, self.c0 + self.boost_rate * prediction
    
    def forward_grad(self, x):
        if len(self.models) == 0:
            batch = x.shape[0]
            c0 = np.repeat(self.c0.detach().cpu().numpy().reshape(1, -1), batch, axis=0)
            return None, torch.Tensor(c0).cuda()
        # at least one model
        middle_feat_cum = None
        prediction = None
        for m in self.models:
            if middle_feat_cum is None:
                middle_feat_cum, prediction = m(x, middle_feat_cum)
            else:
                middle_feat_cum, pred = m(x, middle_feat_cum)
                prediction += pred
        return middle_feat_cum, self.c0 + self.boost_rate * prediction

    @classmethod
    def from_file(cls, path, builder):
        d = torch.load(path)
        net = DynamicNet(d['c0'], d['lr'])
        net.boost_rate = d['boost_rate']
        for stage, m in enumerate(d['models']):
            submod = builder(stage)
            submod.load_state_dict(m)
            net.add(submod)
        return net
    
    def to_file(self, path):
        models = [m.state_dict() for m in self.models]
        d = {'models': models, 'c0': self.c0, 'lr': self.lr, 'boost_rate': self.boost_rate}
        torch.save(d, path)

# model

In [15]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

In [16]:
class MLP_1HL(nn.Module):
    def __init__(self, dim_in, dim_hidden1, dim_hidden2, sparse=False, bn=True):
        super(MLP_1HL, self).__init__()
#         self.in_layer = nn.Linear(dim_in, dim_hidden1)
#         self.out_layer = nn.Linear(dim_hidden1, 206)
#         self.lrelu = nn.LeakyReLU(0.1)
#         self.relu = nn.ReLU()
        self.layer1 = nn.Sequential(
                        nn.Dropout(0.2),
                        nn.Linear(dim_in, dim_hidden1),
                        )
        self.layer2 = nn.Sequential(
                        nn.ReLU(),
#                         nn.BatchNorm1d(dim_hidden1),
#                         nn.Dropout(0.4),
                        nn.Linear(dim_hidden1, 206))
        if bn:
            self.bn = nn.BatchNorm1d(dim_hidden1)
            self.bn2 = nn.BatchNorm1d(dim_in)

    def forward(self, x, lower_f):
        if lower_f is not None:
            x = torch.cat([x, lower_f], dim=1)
            x = self.bn2(x)
        out = self.layer1(x)
        return out, self.layer2(out)

    @classmethod
    def get_model(cls, stage, params):
        if stage == 0:
            dim_in = params["feat_d"]
        else:
            dim_in = params["feat_d"] + params["hidden_size"]
        model = MLP_1HL(dim_in, params["hidden_size"], params["hidden_size"])
        return model
    
class MLP_2HL(nn.Module):
    def __init__(self, dim_in, dim_hidden1, dim_hidden2, sparse=False, bn=True):
        super(MLP_2HL, self).__init__()
        # self.in_layer = SpLinear(dim_in, dim_hidden1) if sparse else nn.Linear(dim_in, dim_hidden1)
        # self.dropout_layer = nn.Dropout(0.4)
        # self.lrelu = nn.LeakyReLU(0.1)
        # self.relu = nn.ReLU()
        # self.hidden_layer = nn.Linear(dim_hidden1, dim_hidden2)
        # self.out_layer = nn.Linear(dim_hidden2, 206)
        # self.bn = nn.BatchNorm1d(dim_hidden1)
        self.bn2 = nn.BatchNorm1d(dim_in)

        self.layer1 = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(dim_in, dim_hidden1),
            nn.ReLU(),
            nn.BatchNorm1d(dim_hidden1),
            nn.Dropout(0.4),
            nn.Linear(dim_hidden1, dim_hidden2)
        )
        self.layer2 = nn.Sequential(
            nn.ReLU(),
#             nn.Dropout(0.4),
            nn.Linear(dim_hidden2, 206)
        )
        
    def forward(self, x, lower_f):
        if lower_f is not None:
            x = torch.cat([x, lower_f], dim=1)
            x = self.bn2(x)
        # out = self.lrelu(self.in_layer(x))
        # out = self.bn(out)
        # out = self.hidden_layer(out)
        middle_feat = self.layer1(x)
        out = self.layer2(middle_feat)
        return middle_feat, out

    @classmethod
    def get_model(cls, stage, params):
        if stage == 0:
            dim_in = params["feat_d"]
        else:
            dim_in = params["feat_d"] + params["hidden_size"]
        model = MLP_2HL(dim_in, params["hidden_size"], params["hidden_size"])
        return model

In [17]:
class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [18]:
def get_optim(params, lr, weight_decay):
    optimizer = optim.Adam(params, lr, weight_decay=weight_decay)
    #optimizer = SGD(params, lr, weight_decay=weight_decay)
    return optimizer

def logloss(net_ensemble, test_loader):
    loss = 0
    total = 0
    loss_f = nn.BCEWithLogitsLoss() # Binary cross entopy loss with logits, reduction=mean by default
    for data in test_loader:
        x = data["x"].cuda()
        y = data["y"].cuda()
        # y = (y + 1) / 2
        with torch.no_grad():
            _, out = net_ensemble.forward(x)
        # out = torch.as_tensor(out, dtype=torch.float32).cuda().view(-1, 1)
        loss += loss_f(out, y)
        total += 1

    return loss / total
device = "cuda" if torch.cuda.is_available() else "cpu"

In [19]:
params = {
          "batch_size": 256,
          "lr": 1e-3,
          "weight_decay": 1e-5,
          "n_folds": 7,
          "early_stopping_steps": 5,
          "hidden_size": 512,
          "boost_rate": 1.0,  # original: 1.0
          "num_nets": 40,  # Number of weak NNs. original: 40
          "epochs_per_stage": 1,  # Number of epochs to learn the Kth model. original: 1
          "correct_epoch": 1,    #  Number of epochs to correct the whole week models original: 1
          "model_order": "second"  # You could put "first" according to the original implemention, but error occurs. original: "second"
          }
params["feat_d"] = len(feature_cols)

In [20]:
train

,sig_id,g-0,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,fold
0,id_000644bb2,1.111702,-0.433285,-0.971725,-0.287840,-1.009546,-1.360720,-0.041833,0.718883,-0.299251,...,0,0,0,0,0,0,0,0,0,1
1,id_000779bfc,0.106780,0.257495,0.086766,1.199177,0.692749,0.353080,0.558392,-0.497809,0.847984,...,0,0,0,0,0,0,0,0,0,2
2,id_000a6266a,0.766810,1.412209,-0.124407,-0.028251,1.492135,0.271884,0.359279,0.038358,1.249253,...,0,0,0,0,0,0,0,0,0,6
3,id_0015fd391,-0.755782,-0.455069,0.765733,2.441521,-0.854377,-2.321686,0.301043,-0.151210,-1.361834,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,-0.469617,0.958032,0.977273,1.445311,-0.864632,-0.346314,-0.225963,-1.026217,0.858427,...,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21943,id_fff8c2444,0.225925,0.202459,-0.361957,-0.364957,0.574001,-0.119274,-0.271844,-0.902164,-0.020179,...,0,0,0,0,0,0,0,0,0,0
21944,id_fffb1ceed,0.196398,-0.255266,-0.799475,-0.720502,0.922380,0.780550,0.509202,-0.583784,-0.031221,...,0,0,0,0,0,0,0,0,0,5
21945,id_fffb70c0c,-1.951184,-0.605409,1.299466,-1.059551,0.855859,-0.274979,-0.731313,0.670101,0.655237,...,0,0,0,0,0,0,0,0,0,3
21946,id_fffcb9e7c,0.803756,0.416740,0.309263,1.069119,-0.020001,0.083853,0.087779,0.571281,-1.174457,...,0,0,0,0,0,0,0,0,0,4


In [21]:
train_targets_scored = pd.read_csv('../input/lish-moa/train_targets_scored.csv')

In [22]:
c0_ = np.log(np.mean(pd.read_csv(DATA_DIR + 'train_targets_scored.csv').drop("sig_id", axis=1).values, axis=0))
def train_fn(seed=0):
    oof = np.zeros((len(train), len(target_feats)))
    predictions = np.zeros((len(test), len(target_feats)))
    
    for fold in range(params["n_folds"]):
        seed_everything(seed)
        
        train_idx = train[train["fold"] != fold].index
        val_idx = train[train["fold"] == fold].index
        
        train_df = train[train["fold"] != fold].reset_index(drop=True)
        val_df = train[train["fold"] == fold].reset_index(drop=True)
        
        x_train = train_df[feature_cols].values
        y_train = train_df[target_feats].values  #
        
        x_val = val_df[feature_cols].values  #
        y_val = val_df[target_feats].values  #
        
        train_ds = MoADataset(x_train, y_train)
        val_ds = MoADataset(x_val, y_val)
        train_loader = torch.utils.data.DataLoader(train_ds, batch_size=params["batch_size"], shuffle=True)
        val_loader = torch.utils.data.DataLoader(val_ds, batch_size=params["batch_size"], shuffle=False)
        
        best_score = np.inf
        val_score = best_score
        best_stage = params["num_nets"] - 1

        c0 = torch.tensor(c0_, dtype=torch.float).to(device)
        net_ensemble = DynamicNet(c0, params["boost_rate"])
        
        loss_f1 = nn.MSELoss(reduction='none')
#         loss_f2 = nn.BCEWithLogitsLoss(reduction='none')
        loss_f2 = SmoothBCEwLogits(smoothing=0.001, reduction="none")
        loss_models = torch.zeros((params["num_nets"], 3))

        all_ensm_losses = []
        all_ensm_losses_te = []
        all_mdl_losses = []
        dynamic_br = []

        lr = params["lr"]
        L2 = params["weight_decay"]        
        
        early_stop = 0
        for stage in range(params["num_nets"]):
            t0 = time.time()
        
            model = MLP_2HL.get_model(stage, params)  # Initialize the model_k: f_k(x), multilayer perception v2
            model.to(device)



            optimizer = get_optim(model.parameters(), lr, L2)
            net_ensemble.to_train() # Set the models in ensemble net to train mode
            stage_mdlloss = []
            for epoch in range(params["epochs_per_stage"]):
                for i, data in enumerate(train_loader):
                    x = data["x"].to(device)
                    y = data["y"].to(device)
                    middle_feat, out = net_ensemble.forward(x)
                    # out = torch.as_tensor(out, dtype=torch.float32).cuda().view(-1, 1)
                    if params["model_order"] == 'first':
                        grad_direction = y / (1.0 + torch.exp(y * out))
                    else:
                        h = 1 / ((1 + torch.exp(y * out)) * (1 + torch.exp(-y * out)))
                        grad_direction = y * (1.0 + torch.exp(-y * out))
                        # out = torch.as_tensor(out)
                        nwtn_weights = (torch.exp(out) + torch.exp(-out)).abs()
                    _, out = model(x, middle_feat)
                    # out = torch.as_tensor(out, dtype=torch.float32).cuda().view(-1, 1)
                    loss = loss_f1(net_ensemble.boost_rate * out, grad_direction)  # T
                    loss = loss * h
                    loss = loss.mean()
                    model.zero_grad()
                    loss.backward()
                    optimizer.step()
                    stage_mdlloss.append(loss.item()) 
                    
            net_ensemble.add(model)
            sml = np.mean(stage_mdlloss)


            stage_loss = []
            lr_scaler = 2
            # fully-corrective step
            if stage != 0:
                # Adjusting corrective step learning rate 
                if stage % 3 == 0:
                    #lr_scaler *= 2
                    lr /= 2
                    # L2 /= 2
                optimizer = get_optim(net_ensemble.parameters(), lr / lr_scaler, L2)
                for _ in range(params["correct_epoch"]):
                    for i, data in enumerate(train_loader):
                        x = data["x"].to(device)
                        y = data["y"].to(device)

                        _, out = net_ensemble.forward_grad(x)
                        # out = torch.as_tensor(out, dtype=torch.float32).cuda().view(-1, 1)
                        # y = (y + 1.0) / 2.0
                        loss = loss_f2(out, y).mean() 
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()
                        stage_loss.append(loss.item())
                        
            sl_te = logloss(net_ensemble, val_loader)  # -----
            # Store dynamic boost rate
            dynamic_br.append(net_ensemble.boost_rate.item())
            # store model
            # net_ensemble.to_file(f"./{fold}FOLD_{seed}_.pth")
            # net_ensemble = DynamicNet.from_file(f"./{fold}FOLD_{seed}_.pth", lambda stage: MLP_2HL.get_model(stage, self.params))

            elapsed_tr = time.time()-t0
            sl = 0
            if stage_loss != []:
                sl = np.mean(stage_loss)

            

            all_ensm_losses.append(sl)
            all_ensm_losses_te.append(sl_te)
            all_mdl_losses.append(sml)
            print(f'Stage - {stage}, training time: {elapsed_tr: .1f} sec, boost rate: {net_ensemble.boost_rate: .4f}, Training Loss: {sl: .5f}, Val Loss: {sl_te: .5f}')


            net_ensemble.to_cuda()
            net_ensemble.to_eval() # Set the models in ensemble net to eval mode
            
            if sl_te < best_score:
                best_score = sl_te
                best_stage = stage
                net_ensemble.to_file(f"./{fold}FOLD_{seed}_.pth")
                early_stop = 0
            else:
                
                early_stop += 1
                

#             test_score = auc_score(net_ensemble, val_loader)
#             print(f'Stage: {stage}, AUC@Val: {val_score:.4f}, AUC@Test: {test_score:.4f}')

#             loss_models[stage, 1], loss_models[stage, 2] = val_score, test_score
            
    
            if early_stop > params["early_stopping_steps"]:
                print("early stopped!")
                break
                
        print(f'Best validation stage: {best_stage}')

        net_ensemble = DynamicNet.from_file(f"./{fold}FOLD_{seed}_.pth", lambda stage: MLP_2HL.get_model(stage, params))
        net_ensemble.to_cuda()
        net_ensemble.to_eval()

        preds = []
        with torch.no_grad():
            for data in val_loader:
                x = data["x"].to(device)
                _, pred = net_ensemble.forward(x)
                preds.append(pred.sigmoid().detach().cpu().numpy())
        oof[val_idx, :] = np.concatenate(preds)

        x_test = test[feature_cols].values
        test_ds = TestDataset(x_test)
        test_loader = torch.utils.data.DataLoader(test_ds, batch_size=params["batch_size"], shuffle=False)

        preds = []
        with torch.no_grad():
            for data in test_loader:
                x = data["x"].to(device)
                _, pred = net_ensemble.forward(x)
                preds.append(pred.sigmoid().detach().cpu().numpy())
        predictions += np.concatenate(preds) / params["n_folds"]
        
    oof = np.clip(oof, 1e-3, 1 - 1e-3)
    predictions = np.clip(predictions, 1e-3, 1 - 1e-3)
    
    train[target_feats] = oof
    test[target_feats] = predictions

    val_results = train_targets_scored.drop(columns=target_feats).merge(train[["sig_id"] + target_feats], on="sig_id", how="left").fillna(0)

    y_true = train_targets_scored[target_feats].values
    y_pred = val_results[target_feats].values

    score = 0
    for i in range(len(target_feats)):
        score_ = log_loss(y_true[:, i], y_pred[:, i])
        score += score_ / len(target_feats)
    print("CV log_loss ", score)

    sub = pd.read_csv('../input/lish-moa/sample_submission.csv')
    sub = sub.drop(columns=target_feats).merge(test[["sig_id"]+target_feats], on="sig_id", how="left").fillna(0)

#         sub = sub.drop(columns=self.target_cols).merge(test_[["sig_id"]+self.target_cols+["cp_time_24", "cp_dose_D2"]], on="sig_id", how="left").fillna(0)
#         sub.loc[:, ["atp-sensitive_potassium_channel_antagonist", "erbb2_inhibitor"]] = 0.000012
#         sub = sub.drop(["cp_time_24", "cp_dose_D2"], axis=1)
    return sub, val_results

In [23]:
sub, grow_oof = train_fn()

Stage - 0, training time:  1.7 sec, boost rate:  1.0000, Training Loss:  0.00000, Val Loss:  0.02009
Stage - 1, training time:  2.5 sec, boost rate:  1.0207, Training Loss:  0.02144, Val Loss:  0.01787
Stage - 2, training time:  2.8 sec, boost rate:  1.0459, Training Loss:  0.02016, Val Loss:  0.01755
Stage - 3, training time:  2.7 sec, boost rate:  1.0623, Training Loss:  0.01903, Val Loss:  0.01752
Stage - 4, training time:  3.0 sec, boost rate:  1.0782, Training Loss:  0.01839, Val Loss:  0.01752
Stage - 5, training time:  3.2 sec, boost rate:  1.0944, Training Loss:  0.01777, Val Loss:  0.01757
Stage - 6, training time:  3.7 sec, boost rate:  1.1031, Training Loss:  0.01677, Val Loss:  0.01771
Stage - 7, training time:  3.7 sec, boost rate:  1.1116, Training Loss:  0.01629, Val Loss:  0.01782
Stage - 8, training time:  3.9 sec, boost rate:  1.1202, Training Loss:  0.01581, Val Loss:  0.01799
Stage - 9, training time:  4.3 sec, boost rate:  1.1247, Training Loss:  0.01511, Val Loss:

In [24]:
sub.to_csv("submission.csv", index=False)